In [7]:
# ✅ Set Gemini API key securely (Google Colab specific)
import os

from dotenv import load_dotenv

load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

from app.prompts import LEARNING_PLANNER, LEARNING_RESEARCHER
from app.schema import LearningPlan

In [8]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.5, max_retries=2, api_key=gemini_api_key)
struct_bound = llm.with_structured_output(LearningPlan)

system_message = LEARNING_PLANNER.format(plan=None)

response = struct_bound.invoke([("system", system_message), ("user", "how to be a pro drummer")])

print(response)


## research agent

response = llm.invoke([("system", LEARNING_RESEARCHER), ("user", "how to be a pro drummer")])

print(response)

topic='Drumming' duration_weeks=4 weekly_plans=[WeekPlan(week_number=1, focus='Mastering fundamental drumming techniques and timing.', activities=[Activity(description='Practice basic rudiments (single stroke roll, double stroke roll, paradiddle) on a practice pad.', frequency='Daily'), Activity(description='Work on timing and tempo with a metronome.', frequency='Daily'), Activity(description='Listen to recordings of famous drummers and transcribe their beats.', frequency='3x/week')], resources=[Resource(name="Alfred's Drum Method, Book 1", type='book', url=None), Resource(name='Vic Firth practice pad', type='other', url=None), Resource(name='Metronome app', type='app', url=None)], checkpoint='Record yourself playing a basic rock beat with consistent tempo and clean execution.'), WeekPlan(week_number=2, focus='Developing drum kit coordination and genre versatility.', activities=[Activity(description='Apply rudiments to the drum kit (snare, toms, cymbals).', frequency='Daily'), Activity

In [9]:
# ✅ 🔧 MISSING FUNCTION: generate_learning_path()
def generate_learning_path(topic):
    response = struct_bound.invoke([("system", system_message), ("user", topic)])
    return response.model_dump_json(indent=2)


# ✅ Optional test to see if Gemini works (can comment out later)
print(generate_learning_path("Python"))

{
  "topic": "Python",
  "duration_weeks": 4,
  "weekly_plans": [
    {
      "week_number": 1,
      "focus": "Understanding basic Python syntax and data types",
      "activities": [
        {
          "description": "Read introductory chapter on Python syntax and data types",
          "frequency": "Once"
        },
        {
          "description": "Complete online tutorial on variables and operators",
          "frequency": "3 times"
        },
        {
          "description": "Practice writing simple Python scripts",
          "frequency": "Daily"
        }
      ],
      "resources": [
        {
          "name": "Official Python Tutorial",
          "type": "documentation",
          "url": "https://docs.python.org/3/tutorial/"
        },
        {
          "name": "Codecademy Python Course",
          "type": "app",
          "url": "https://www.codecademy.com/learn/learn-python-3"
        }
      ],
      "checkpoint": "Write a Python script that takes user input and per

In [ ]:
# ✅ Gradio interface
import gradio as gr


def chatbot(topic):
    if not topic.strip():
        return "Please enter a topic."
    return generate_learning_path(topic)


def download_learning_path(topic):
    plan = generate_learning_path(topic)
    filename = f"{topic}_learning_path.txt"
    with open(filename, "w") as f:
        f.write(plan)
    return filename


with gr.Blocks() as demo:
    gr.Markdown("## 📘 Gemini AI Learning Path Generator")

    with gr.Row():
        topic_input = gr.Textbox(label="Enter a Topic")
        generate_btn = gr.Button("Generate Learning Path")

    output = gr.Textbox(label="Learning Path", lines=20)
    download_btn = gr.Button("Download")

    generate_btn.click(chatbot, inputs=topic_input, outputs=output)
    download_btn.click(download_learning_path, inputs=topic_input, outputs=None)

demo.launch(share=True, debug=True)